In [49]:
include("build_profiles.jl")

build_all_ngrams_profiles (generic function with 1 method)

In [9]:
D,h = build_ngrams_profile("zho")

# zho


total vocab: 3838.139k; tokens: 4.300675M; dataset ratio: [0.4736709935068332, 0.5263290064931667](vocab) [0.6262618895235783, 0.3737381104764217](token)
Profile size: 100.0k(3.0%) @freq=17.0; [min(1586276, 172656)]

(Pair{Vector{UInt8}, Float64}[[0xe5] => 2.6446336401941632, [0xe6] => 1.8244921518987316, [0x20] => 1.7450992082830203, [0xe4] => 1.3984936823149554, [0xe7] => 1.202039853064266, [0xe8] => 0.9999424974642741, [0xe9] => 0.6879887538425971, [0xb8] => 0.6824734722381373, [0xba] => 0.6253072948276369, [0x88] => 0.5582100762980283  …  [0x70, 0x65, 0x6e] => 0.0001666471402579273, [0x75, 0x72, 0x69] => 0.0001666471402579273, [0x81, 0xb8, 0x20] => 0.0001666471402579273, [0x83, 0x91, 0xe5] => 0.0001666471402579273, [0x83, 0xe7, 0xba] => 0.0001666471402579273, [0x84, 0xe8, 0x92] => 0.0001666471402579273, [0x88, 0x8f, 0xe6] => 0.0001666471402579273, [0x89, 0x88, 0xe9] => 0.0001666471402579273, [0x89, 0xe9, 0xa6] => 0.0001666471402579273, [0x8d, 0xb0, 0xe7] => 0.0001666471402579273], [28.736326861175957, 28.271466720101195, 27.806606579013796, 27.34174643778948, 26.876886296411964, 26.412026154865217, 25.947445585720693])

In [54]:
@time build_all_ngrams_profiles(LANGS[1:3], "ngrams")

# ara


total vocab: 319.514k; tokens: 0.468327M; dataset ratio: [0.479564492331213, 0.520435507668787](vocab) [0.5908803438457672, 0.4091196561542329](token)
Profile size: 27.918k(9.0%) @freq=73.0; [min(27918, 98005)]
 19.083182 seconds (40.88 M allocations: 3.177 GiB, 3.23% gc time)


# bel


total vocab: 256.874k; tokens: 0.352388M; dataset ratio: [0.338601200948954, 0.661398799051046](vocab) [0.2994345997785773, 0.7005654002214228](token)
Profile size: 21.191k(8.0%) @freq=89.0; [min(21191, 58923)]
 18.814852 seconds (31.37 M allocations: 2.606 GiB, 1.17% gc time)


# ben


total vocab: 104.597k; tokens: 0.12752M; dataset ratio: [0.18727258469259725, 0.8127274153074028](vocab) [0.18979096735353854, 0.8102090326464615](token)
Profile size: 2.917k(3.0%) @freq=1183.0; [min(2917, 17771)]
 19.397841 seconds (47.06 M allocations: 3.510 GiB, 3.79% gc time)


 57.397326 seconds (120.52 M allocations: 9.357 GiB, 2.74% gc time)


In [12]:
D,h = build_ngrams_profile("yid")

# yid


total vocab: 135.062k; tokens: 0.1862M; dataset ratio: [0.33967239527389903, 0.660327604726101](vocab) [0.32930428350561436, 0.6706957164943856](token)
Profile size: 10.008k(7.0%) @freq=162.0; [min(10008, 41673)]


(Pair{Vector{UInt8}, Float64}[[0xd7] => 12.132796741701616, [0x20] => 3.789403273495392, [0x20, 0xd7] => 3.5162187256097748, [0x99] => 1.5591632268892548, [0xd7, 0x99] => 1.5591632268892548, [0x90] => 1.4649511189375592, [0xd7, 0x90] => 1.4649511189375592, [0x99, 0xd7] => 1.3667675852862382, [0xd7, 0x99, 0xd7] => 1.3667675852862382, [0xa2] => 1.1506468924279525  …  [0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575, [0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575, [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575, [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575, [0x95, 0xd7, 0x99, 0xd7, 0x99, 0xd7, 0x9c] => 0.0016152803763646896, [0x90, 0xd7, 0x9b, 0xd7, 0x98, 0xd7, 0xa2] => 0.001614766205078347, [0xa9, 0xd7, 0x98, 0xd7, 0x90, 0xd7, 0xa8] => 0.001613737862505662, [0xa2, 0x20, 0xd7, 0x93, 0xd7, 0x99, 0x20] => 0.0016132236912193194, [0x93, 0xd7, 0x96, 0xd7, 0xa2, 0xd7, 0xa8] => 0.0016126407888742012, [0x9c, 0xd7, 0x99, 0xd7, 0x93] => 0.0

In [50]:
dump_ngrams(D, "nt.txt", head=h)

In [51]:
D2,h2 = load_ngrams("nt.txt")

(Pair{Vector{UInt8}, Float64}[[0xd7] => 12.132796741701616, [0x20] => 3.789403273495392, [0x20, 0xd7] => 3.5162187256097748, [0x99] => 1.5591632268892548, [0xd7, 0x99] => 1.5591632268892548, [0x90] => 1.4649511189375592, [0xd7, 0x90] => 1.4649511189375592, [0x99, 0xd7] => 1.3667675852862382, [0xd7, 0x99, 0xd7] => 1.3667675852862382, [0xa2] => 1.1506468924279525  …  [0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575, [0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575, [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575, [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575, [0x95, 0xd7, 0x99, 0xd7, 0x99, 0xd7, 0x9c] => 0.0016152803763646896, [0x90, 0xd7, 0x9b, 0xd7, 0x98, 0xd7, 0xa2] => 0.001614766205078347, [0xa9, 0xd7, 0x98, 0xd7, 0x90, 0xd7, 0xa8] => 0.001613737862505662, [0xa2, 0x20, 0xd7, 0x93, 0xd7, 0x99, 0x20] => 0.0016132236912193194, [0x93, 0xd7, 0x96, 0xd7, 0xa2, 0xd7, 0xa8] => 0.0016126407888742012, [0x9c, 0xd7, 0x99, 0xd7, 0x93] => 0.0

In [52]:
D==D2, h==h2

(true, true)

In [56]:
match(r"abc", "ksabcab")

RegexMatch("abc")

In [ ]:
str = "abcdef"
last_str = "ddcdeg"
function lcs_zip(str, refer)
    rg1, rg2 = lcs(refer, str)
    if length(rg1) > 2
        b1, e1 = first(rg1), last(rg1)
        b2, e2 = first(rg2), last(rg2)
        if e1 < 26
            bc = Char(Int('A') + b1 - 1)
            ec = Char(Int('A') + e1 - 1)
            last_str = string(last_str[1:b2-1], bc, ec, last_str[e2+1:end])
        end
    end
    last_str
end

In [ ]:
lcs_zip(str, last_str)

In [107]:
string('a', "ff")

"aff"

In [108]:
function lcs(s1::AbstractString, s2::AbstractString)
    m, n = length(s1), length(s2)
    len, pos1, pos2 = 0, 0, 0
    dp = zeros(Int, m+1, n+1)
    for i in 1:m
        for j in 1:n
            if s1[i] == s2[j]
                dp[i+1, j+1] = dp[i, j] + 1
                if dp[i+1, j+1] > len
                    len = dp[i+1, j+1]
                    pos1 = i
                    pos2 = j
                end
            end
        end
    end
    return pos1 - len + 1:pos1, pos2 - len + 1:pos2
end


lcs (generic function with 1 method)

In [109]:
lcs("---","")

(1:0, 1:0)

In [110]:
length(1:0)

0

In [111]:
match(r"abdc", "ksabcab")

In [112]:
Char(Int('A') + 26 - 1)

'Z': ASCII/Unicode U+005A (category Lu: Letter, uppercase)

In [113]:
Char.(Int('A') .+ findfirst("abc", "ksabcab"))

3-element Vector{Char}:
 'D': ASCII/Unicode U+0044 (category Lu: Letter, uppercase)
 'E': ASCII/Unicode U+0045 (category Lu: Letter, uppercase)
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)

In [114]:
splice!

splice! (generic function with 9 methods)

In [115]:
D2

10008-element Vector{Pair{Vector{UInt8}, Float64}}:
                                     [0xd7] => 12.132796741701616
                                     [0x20] => 3.789403273495392
                               [0x20, 0xd7] => 3.5162187256097748
                                     [0x99] => 1.5591632268892548
                               [0xd7, 0x99] => 1.5591632268892548
                                     [0x90] => 1.4649511189375592
                               [0xd7, 0x90] => 1.4649511189375592
                               [0x99, 0xd7] => 1.3667675852862382
                         [0xd7, 0x99, 0xd7] => 1.3667675852862382
                                     [0xa2] => 1.1506468924279525
                                            ⋮
       [0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575
       [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575
 [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575
 [0x95, 0xd7, 0x99, 0xd7, 0x99, 0xd7

In [116]:
G = D
sort(collect(G), by=i->(-last(i), length(first(i)), first(i)))

10008-element Vector{Pair{Vector{UInt8}, Float64}}:
                                     [0xd7] => 12.132796741701616
                                     [0x20] => 3.789403273495392
                               [0x20, 0xd7] => 3.5162187256097748
                                     [0x99] => 1.5591632268892548
                               [0xd7, 0x99] => 1.5591632268892548
                                     [0x90] => 1.4649511189375592
                               [0xd7, 0x90] => 1.4649511189375592
                               [0x99, 0xd7] => 1.3667675852862382
                         [0xd7, 0x99, 0xd7] => 1.3667675852862382
                                     [0xa2] => 1.1506468924279525
                                            ⋮
       [0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575
       [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575
 [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575
 [0x95, 0xd7, 0x99, 0xd7, 0x99, 0xd7

In [117]:
build_ngrams_profile("yid")

# yid


total vocab: 135.062k; tokens: 0.1862M; dataset ratio: [0.33967239527389903, 0.660327604726101](vocab) [0.32930428350561436, 0.6706957164943856](token)
Profile size: 10.008k(7.0%) @freq=162.0; [min(10008, 41673)]


(Pair{Vector{UInt8}, Float64}[[0xd7] => 12.132796741701616, [0x20] => 3.789403273495392, [0x20, 0xd7] => 3.5162187256097748, [0x99] => 1.5591632268892548, [0xd7, 0x99] => 1.5591632268892548, [0x90] => 1.4649511189375592, [0xd7, 0x90] => 1.4649511189375592, [0x99, 0xd7] => 1.3667675852862382, [0xd7, 0x99, 0xd7] => 1.3667675852862382, [0xa2] => 1.1506468924279525  …  [0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575, [0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575, [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575, [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575, [0x95, 0xd7, 0x99, 0xd7, 0x99, 0xd7, 0x9c] => 0.0016152803763646896, [0x90, 0xd7, 0x9b, 0xd7, 0x98, 0xd7, 0xa2] => 0.001614766205078347, [0xa9, 0xd7, 0x98, 0xd7, 0x90, 0xd7, 0xa8] => 0.001613737862505662, [0xa2, 0x20, 0xd7, 0x93, 0xd7, 0x99, 0x20] => 0.0016132236912193194, [0x93, 0xd7, 0x96, 0xd7, 0xa2, 0xd7, 0xa8] => 0.0016126407888742012, [0x9c, 0xd7, 0x99, 0xd7, 0x93] => 0.0

In [118]:
build_ngrams_profile("zho")
build_ngrams_profile("yid")
build_ngrams_profile("ina")


# zho


total vocab: 3838.139k; tokens: 4.300675M; dataset ratio: [0.4736709935068332, 0.5263290064931667](vocab) [0.6262618895235783, 0.3737381104764217](token)
Profile size: 100.0k(3.0%) @freq=17.0; [min(1586276, 172656)]


# yid

In [ ]:
build_ngrams_profile("zho", ngram=5)
build_ngrams_profile("yid", ngram=5)
build_ngrams_profile("ina", ngram=5)
;

In [ ]:
build_ngrams_profile("zho", ratio=0.7)
build_ngrams_profile("yid", ratio=0.7)
build_ngrams_profile("ina", ratio=0.7)


In [ ]:
build_ngrams_profile("zho", ratio=0.7, ngram=5)
build_ngrams_profile("yid", ratio=0.7, ngram=5)
build_ngrams_profile("ina", ratio=0.7, ngram=5)
